In [263]:
import pandas as pd
import glob

In [264]:
files = glob.glob('./results/*/*')

In [265]:
df = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    df = df.append(d, ignore_index=True)

In [266]:
classes = ['risky', 'safe', 'unknown']
train_Y = df[classes] 
train_X = df.drop(classes, axis=1)

In [267]:
train_Y.head()

,risky,safe,unknown
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [268]:
len(train_X.columns)

32

In [269]:
import numpy as np
X = (train_X - train_X.min()) / (train_X.max() - train_X.min() + 000000000000000000000000000000.1)
#X = X[['T_VARIABLE', 'T_ARRAY', 'T_IF', 'T_ELSE', 'T_FOREACH', 'T_BOOLEAN_AND', 'T_BOOLEAN_OR', 'T_CONCAT_EQUAL','T_DOUBLE_ARROW','T_EMPTY', 'T_RETURN', 'T_FUNCTION']]
X.sample(10)

,T_VARIABLE,T_SWITCH,T_CASE,T_ARRAY,T_FUNCTION,T_IF,T_ELSE,T_ENDIF,T_FOR,T_ENDFOR,...,T_OBJECT_OPERATOR,T_TRY,T_THROW,T_CATCH,T_BOOLEAN_AND,T_BOOLEAN_OR,T_CONCAT_EQUAL,T_DOUBLE_ARROW,T_EMPTY,T_RETURN
104,0.240089,0.0,0.0,0.000000,0.0,0.116959,0.000000,0.0,0.0,0.0,...,0.108617,0.0,0.0,0.0,0.392157,0.000000,0.0,0.147601,0.09009,0.000000
43,0.251256,0.0,0.0,0.000000,0.0,0.292398,0.000000,0.0,0.0,0.0,...,0.181028,0.0,0.0,0.0,0.392157,0.000000,0.0,0.110701,0.18018,0.322581
24,0.050251,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.028965,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.322581
6,0.083752,0.0,0.0,0.000000,0.0,0.116959,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.221402,0.00000,0.000000
54,0.279174,0.0,0.0,0.000000,0.0,0.643275,0.243902,0.0,0.0,0.0,...,0.094135,0.0,0.0,0.0,0.392157,0.000000,0.0,0.036900,0.27027,0.000000
38,0.429927,0.0,0.0,0.000000,0.0,0.643275,0.000000,0.0,0.0,0.0,...,0.101376,0.0,0.0,0.0,0.588235,0.909091,0.0,0.184502,0.27027,0.322581
48,0.106086,0.0,0.0,0.000000,0.0,0.058480,0.000000,0.0,0.0,0.0,...,0.036206,0.0,0.0,0.0,0.196078,0.000000,0.0,0.221402,0.00000,0.000000
67,0.178671,0.0,0.0,0.000000,0.0,0.409357,0.487805,0.0,0.0,0.0,...,0.115858,0.0,0.0,0.0,0.196078,0.000000,0.0,0.000000,0.27027,0.000000
28,0.106086,0.0,0.0,0.000000,0.0,0.175439,0.000000,0.0,0.0,0.0,...,0.079652,0.0,0.0,0.0,0.196078,0.000000,0.0,0.110701,0.09009,0.322581
82,0.195422,0.0,0.0,0.952381,0.0,0.292398,0.000000,0.0,0.0,0.0,...,0.072411,0.0,0.0,0.0,0.000000,0.909091,0.0,0.147601,0.00000,0.000000


In [270]:
import keras

In [271]:
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD
import keras.backend as K
from keras.callbacks import ModelCheckpoint

In [287]:
K.clear_session()
model = Sequential()
model.add(Dense(512, input_shape=(len(X.columns),)))
model.add(Dropout(.2))
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Dense(128))
#model.add(Dropout(.2))
model.add(Dense(64))
model.add(Dense(3))

opt = SGD(lr=0.0001)
#opt = Adam(lr=0.0001)

model.compile(opt, loss='categorical_crossentropy', metrics=['acc'])

In [288]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 195       
Total para

In [329]:
# Create checkpoint
filepath="weights/weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# load weights
#model.load_weights("weights-improvement-06-0.49.hdf5")

# Fit the model
model.fit(X, train_Y, epochs=100, shuffle=True, callbacks=callbacks_list)

Epoch 1/100
123/123 [==============================] - 0s 1ms/step - loss: 0.8901 - acc: 0.4065

Epoch 00001: acc improved from -inf to 0.40650, saving model to weights/weights-improvement-01-0.41.hdf5
Epoch 2/100
123/123 [==============================] - 0s 1ms/step - loss: 0.8886 - acc: 0.4959

Epoch 00002: acc improved from 0.40650 to 0.49593, saving model to weights/weights-improvement-02-0.50.hdf5
Epoch 3/100
123/123 [==============================] - 0s 840us/step - loss: 0.8894 - acc: 0.3740

Epoch 00003: acc did not improve from 0.49593
Epoch 4/100
123/123 [==============================] - 0s 1ms/step - loss: 0.8932 - acc: 0.4553

Epoch 00004: acc did not improve from 0.49593
Epoch 5/100
123/123 [==============================] - 0s 976us/step - loss: 0.8926 - acc: 0.4146

Epoch 00005: acc did not improve from 0.49593
Epoch 6/100
123/123 [==============================] - 0s 735us/step - loss: 0.8855 - acc: 0.4472

Epoch 00006: acc did not improve from 0.49593
Epoch 7/100
123

In [324]:
test_files = files = glob.glob('./previsions/*')
test_df = pd.DataFrame()
for f in test_files:
    d = pd.read_csv(f)
    test_df = test_df.append(d, ignore_index=True)

In [325]:
X_test = (test_df - test_df.min()) / (test_df.max() - test_df.min() + 000000000000000000000000000000.1)
#X = X[['T_VARIABLE', 'T_ARRAY', 'T_IF', 'T_ELSE', 'T_FOREACH', 'T_BOOLEAN_AND', 'T_BOOLEAN_OR', 'T_CONCAT_EQUAL','T_DOUBLE_ARROW','T_EMPTY', 'T_RETURN', 'T_FUNCTION']]
X_test

,T_VARIABLE,T_SWITCH,T_CASE,T_ARRAY,T_FUNCTION,T_IF,T_ELSE,T_ENDIF,T_FOR,T_ENDFOR,...,T_OBJECT_OPERATOR,T_TRY,T_THROW,T_CATCH,T_BOOLEAN_AND,T_BOOLEAN_OR,T_CONCAT_EQUAL,T_DOUBLE_ARROW,T_EMPTY,T_RETURN
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [326]:
prediction = model.predict(X_test)

In [327]:
print("Prediction:", prediction[0])

Prediction: [0.23894009 0.21016829 0.12881328]


In [328]:
classes[np.argmax(prediction)], prediction[0][np.argmax(prediction)]*100, '%'

('risky', 23.894008994102478, '%')